# Fiftyone

The open-source tool for building high-quality datasets and computer vision models.

<img src="https://voxel51.com/docs/fiftyone/_static/images/homepage_curate.gif" alt="drawing" width="500"/>

Install with pip in a python virtual environment (conda recommended).
```console
pip install fiftyone
```
Detailed instructions here: https://voxel51.com/docs/fiftyone/getting_started/install.html

In [1]:
import fiftyone as fo
fo.list_datasets()

['tmp-dataset']

In [2]:
name = "tmp-dataset"
dataset_dir = "./dataset/"

if name in fo.list_datasets():
    fo.delete_dataset(name)

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.CVATVideoDataset,
    name=name,
)

# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

 100% |███████████████████| 16/16 [18.6s elapsed, 0s remaining, 0.8 samples/s]     
Name:        tmp-dataset
Media type:  video
Num samples: 16
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.VideoMetadata)
Frame fields:
    id:           fiftyone.core.fields.ObjectIdField
    frame_number: fiftyone.core.fields.FrameNumberField
    detections:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
[<Sample: {
    'id': '63b802fadc1ab3fd20bd10b9',
    'media_type': 'video',
    'filepath': '/Users/kevinserrano/GitHub/JKU_C0_CV/dataset/data/video_012.mp4',
    'tags': [],
    'metadata': None,
    'frames': <Frames: 733>,
}>, <Sample: {
    'id': '63b802fbdc1ab3fd20bd165c',
    'media_type': 'video',
    'filepat

In [3]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


In [4]:
import fiftyone.utils.random as four

four.random_split(dataset, {"train": 0.7, "val": 0.3}, seed=51)
session.view = dataset.view()

In [5]:
# few examples on views
view = dataset.match_tags("val")
session.view = view
# session.show()

In [6]:
# convert video to frames
import fiftyone.core.video as fcv
frames_dataset = fcv.make_frames_dataset(
    dataset,
    sample_frames=True,
    fps=2,
)

Computing metadata...
 100% |███████████████████| 16/16 [246.7ms elapsed, 0s remaining, 64.8 samples/s] 
Setting 1489 frame filepaths on the input collection that exist on disk but are not recorded on the dataset


In [7]:
session.view = frames_dataset.view()

In [9]:
# export dataset for yolo training
classes = frames_dataset.distinct("detections.detections.label")

# export dir
export_dir = "./yolov5/CV-C0-v0"

# YOLOv5 format supports splits
for split in ["train", "val"]:
    view = frames_dataset.match_tags(split)
    view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        label_field="detections",
        classes=classes,
    )

 100% |███████████████| 1072/1072 [2.2s elapsed, 0s remaining, 496.0 samples/s]      
Directory './yolov5/CV-C0-v0' already exists; export will be merged with existing files
 100% |█████████████████| 417/417 [796.7ms elapsed, 0s remaining, 523.4 samples/s]      
